In [1]:
#####========================================= Import Libraries ============================================================

import pandas as pd
import numpy as np
import scipy
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.util import bigrams 
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import warnings
warnings.filterwarnings("ignore")

In [2]:
####==================================Import and read the dataset===========================================================

# Original dataset
df_orig = pd.read_csv("DataSet.csv")
df_orig = pd.DataFrame(df_orig)

# replicate for work
df = df_orig

#### =============== Unprocessed data ===========================
print(df.shape)
df.head()

(17880, 18)


,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,in_balanced_dataset
0,Marketing Intern,"US, NY, New York",Marketing,NaN,"<h3>We're Food52, and we've created a groundbr...","<p>Food52, a fast-growing, James Beard Award-w...",<ul>\r\n<li>Experience with content management...,NaN,f,t,f,Other,Internship,NaN,NaN,Marketing,f,f
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"<h3>90 Seconds, the worlds Cloud Video Product...",<p>Organised - Focused - Vibrant - Awesome!<br...,<p><b>What we expect from you:</b></p>\r\n<p>Y...,<h3><b>What you will get from us</b></h3>\r\n<...,f,t,f,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,f,f
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,<h3></h3>\r\n<p>Valor Services provides Workfo...,"<p>Our client, located in Houston, is actively...",<ul>\r\n<li>Implement pre-commissioning and co...,NaN,f,t,f,NaN,NaN,NaN,NaN,NaN,f,f
3,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,<p>Our passion for improving quality of life t...,<p><b>THE COMPANY: ESRI – Environmental System...,<ul>\r\n<li>\r\n<b>EDUCATION: </b>Bachelor’s o...,<p>Our culture is anything but corporate—we ha...,f,t,f,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,f,f
4,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,<p>SpotSource Solutions LLC is a Global Human ...,<p><b>JOB TITLE:</b> Itemization Review Manage...,<p><b>QUALIFICATIONS:</b></p>\r\n<ul>\r\n<li>R...,<p>Full Benefits Offered</p>,f,t,t,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,f,f


In [3]:
####====================================== Data Preprocessing: ============================================================

##### Steps Involved:

#1) Remove HTML Tags 

#2) Tokenization

#3) Remove Punctuations

#4) Lemmatization

#5) Remove Stopword

### Remove HTML Tags

In [4]:
import re

def remove_html_tags(text):
    return re.sub('<[^<]+?>', '', str(text))


df.company_profile=df.company_profile.apply(remove_html_tags).str.replace('\r\n', '').str.lower()
df.description=df.description.apply(remove_html_tags).str.replace('\r\n', '').str.lower()
df.requirements=df.requirements.apply(remove_html_tags).str.replace('\r\n', '').str.lower()
df.benefits=df.benefits.apply(remove_html_tags).str.replace('\r\n', '').str.lower()
df.head()

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,in_balanced_dataset
0,Marketing Intern,"US, NY, New York",Marketing,NaN,"we're food52, and we've created a groundbreaki...","food52, a fast-growing, james beard award-winn...",experience with content management systems a m...,nan,f,t,f,Other,Internship,NaN,NaN,Marketing,f,f
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 seconds, the worlds cloud video production ...",organised - focused - vibrant - awesome!do you...,what we expect from you:your key responsibilit...,what you will get from usthrough being part of...,f,t,f,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,f,f
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,valor services provides workforce solutions th...,"our client, located in houston, is actively se...",implement pre-commissioning and commissioning ...,nan,f,t,f,NaN,NaN,NaN,NaN,NaN,f,f
3,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,our passion for improving quality of life thro...,the company: esri – environmental systems rese...,"education: bachelor’s or master’s in gis, busi...",our culture is anything but corporate—we have ...,f,t,f,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,f,f
4,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,spotsource solutions llc is a global human cap...,job title: itemization review managerlocation:...,qualifications:rn license in the state of texa...,full benefits offered,f,t,t,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,f,f


### Tokenization and Remove Punctuations 


In [5]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
df['company_profile'] = df['company_profile'].apply(lambda x: tokenizer.tokenize(x))
df['description'] = df['description'].apply(lambda x: tokenizer.tokenize(x))
df['requirements'] = df['requirements'].apply(lambda x: tokenizer.tokenize(x))
df['benefits'] = df['benefits'].apply(lambda x: tokenizer.tokenize(x))

df.head()

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,in_balanced_dataset
0,Marketing Intern,"US, NY, New York",Marketing,NaN,"[we, re, food52, and, we, ve, created, a, grou...","[food52, a, fast, growing, james, beard, award...","[experience, with, content, management, system...",[nan],f,t,f,Other,Internship,NaN,NaN,Marketing,f,f
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"[90, seconds, the, worlds, cloud, video, produ...","[organised, focused, vibrant, awesome, do, you...","[what, we, expect, from, you, your, key, respo...","[what, you, will, get, from, usthrough, being,...",f,t,f,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,f,f
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,"[valor, services, provides, workforce, solutio...","[our, client, located, in, houston, is, active...","[implement, pre, commissioning, and, commissio...",[nan],f,t,f,NaN,NaN,NaN,NaN,NaN,f,f
3,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,"[our, passion, for, improving, quality, of, li...","[the, company, esri, environmental, systems, r...","[education, bachelor, s, or, master, s, in, gi...","[our, culture, is, anything, but, corporate, w...",f,t,f,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,f,f
4,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,"[spotsource, solutions, llc, is, a, global, hu...","[job, title, itemization, review, managerlocat...","[qualifications, rn, license, in, the, state, ...","[full, benefits, offered]",f,t,t,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,f,f


### Lemmatization of Tokens and Stopwords Removal

In [6]:
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
stop = stopwords.words('english')

from nltk.stem import WordNetLemmatizer 
wordnet_lemmatizer = WordNetLemmatizer() 

def lemma(data):
    
    lemmatized_data = [wordnet_lemmatizer.lemmatize(word) for word in data]
    clean_data = [w for w in lemmatized_data if w not in stop]
    
    return clean_data

df['company_profile'] = df['company_profile'].apply(lambda x: lemma(x))
df['description'] = df['description'].apply(lambda x: lemma(x))
df['requirements'] = df['requirements'].apply(lambda x: lemma(x))
df['benefits'] = df['benefits'].apply(lambda x: lemma(x))


df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vashiv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,in_balanced_dataset
0,Marketing Intern,"US, NY, New York",Marketing,NaN,"[food52, created, groundbreaking, award, winni...","[food52, fast, growing, james, beard, award, w...","[experience, content, management, system, majo...",[nan],f,t,f,Other,Internship,NaN,NaN,Marketing,f,f
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"[90, second, world, cloud, video, production, ...","[organised, focused, vibrant, awesome, passion...","[expect, key, responsibility, communicate, cli...","[get, usthrough, part, 90, second, team, gain,...",f,t,f,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,f,f
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,"[valor, service, provides, workforce, solution...","[client, located, houston, actively, seeking, ...","[implement, pre, commissioning, commissioning,...",[nan],f,t,f,NaN,NaN,NaN,NaN,NaN,f,f
3,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,"[passion, improving, quality, life, geography,...","[company, esri, environmental, system, researc...","[education, bachelor, master, gi, business, ad...","[culture, anything, corporate, collaborative, ...",f,t,f,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,f,f
4,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,"[spotsource, solution, llc, global, human, cap...","[job, title, itemization, review, managerlocat...","[qualification, rn, license, state, texasdiplo...","[full, benefit, offered]",f,t,t,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,f,f


In [7]:
df["company_profile"].head(5)

0    [food52, created, groundbreaking, award, winni...
1    [90, second, world, cloud, video, production, ...
2    [valor, service, provides, workforce, solution...
3    [passion, improving, quality, life, geography,...
4    [spotsource, solution, llc, global, human, cap...
Name: company_profile, dtype: object